# Spin-1/2 1-D Heisenberg model

## Overview

This notebook will give an example how to use Renormalizer to calculate the ground state energy of the open boundary spin 1/2 1-D Heisenberg model.

$$
H = J \sum_i [S_i^z S_{i+1}^z + \frac{1}{2}(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)]
$$

Exact result via Bethe Anstatz:

|$L$ |      $E/J$|
|----|-----------|
|16  |-6.9117371455749|
|24  |-10.4537857604096|
|32  |-13.9973156182243|
|48  |-21.0859563143863|
|64  |-28.1754248597421|


## Setup

In [1]:
from renormalizer import Model, Op, BasisHalfSpin,  Mps, Mpo, optimize_mps

2024-06-27 07:33:56,731[INFO] Use NumPy as backend


2024-06-27 07:33:56,731[INFO] numpy random seed is 9012


2024-06-27 07:33:56,732[INFO] random seed is 1092


2024-06-27 07:33:56,744[INFO] Git Commit Hash: f8fda66a146b44930935f06cae2608b9df990075


2024-06-27 07:33:56,745[INFO] use 64 bits


## Define the Model
In Renormalizer, models are defined by the Hamiltonian terms and the a list of basis sets. The basis also defines the ordering in DMRG.

The spin operators can be represented by Pauli operators
$$
S^+ = \sigma^+ 
$$
$$
S^- = \sigma^- 
$$
$$
S^{\{x,y,z\}} = \frac{1}{2} \sigma^{\{x,y,z\}}
$$

In [2]:
# define the # of spins
nspin = 32

# define the model

ham_terms = []
for ispin in range(nspin-1):
    op1 = Op("sigma_z sigma_z", [ispin, ispin+1], 1.0/4)
    op2 = Op("sigma_+ sigma_-", [ispin, ispin+1], 1.0/2)
    op3 = Op("sigma_- sigma_+", [ispin, ispin+1], 1.0/2)
    ham_terms.extend([op1, op2, op3])

# set the spin order and local basis
basis = [BasisHalfSpin(i) for i in range(nspin)]
# construct Hamiltonian MPO
model = Model(basis, ham_terms)
mpo = Mpo(model)
print(f"mpo_bond_dims:{mpo.bond_dims}")

2024-06-27 07:33:56,853[DEBUG] # of operator terms: 93


2024-06-27 07:33:56,853[DEBUG] symbolic mpo algorithm: qr


2024-06-27 07:33:56,854[DEBUG] Input operator terms: 93


2024-06-27 07:33:56,879[DEBUG] After combination of the same terms: 93


mpo_bond_dims:[1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1]


## DMRG Sweeps

In [3]:
# set the sweep paramter
M = 30
procedure = [[M, 0.2], [M, 0], [M, 0], [M,0], [M,0]]

# initialize a random MPS
qntot = 0
mps = Mps.random(model, qntot, M)

mps.optimize_config.procedure = procedure
mps.optimize_config.method = "2site"

# optimize MPS
energies, _ = optimize_mps(mps.copy(), mpo)
print("gs energy:", min(energies))

2024-06-27 07:33:56,948[INFO] optimization method: 2site


2024-06-27 07:33:56,949[INFO] e_rtol: 1e-06


2024-06-27 07:33:56,950[INFO] e_atol: 1e-08


2024-06-27 07:33:56,951[INFO] procedure: [[30, 0.2], [30, 0], [30, 0], [30, 0], [30, 0]]


2024-06-27 07:33:56,982[DEBUG] isweep: 0


...

2024-06-27 07:34:02,911[DEBUG] energy: -13.997315007427405


2024-06-27 07:34:02,914[DEBUG] 3 sweeps are finished, lowest energy = -13.997315090211648


2024-06-27 07:34:02,917[INFO] DMRG has converged!


2024-06-27 07:34:02,951[INFO] mps current size: 329.7KiB, Matrix product bond dim:[1, 2, 4, 8, 16, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 16, 8, 4, 2, 1]


gs energy: -13.997315090211648
